In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Trialwise Decoding

In this example, we will use a convolutional neural network on the [Physiobank EEG Motor Movement/Imagery Dataset](https://www.physionet.org/physiobank/database/eegmmidb/) to decode two classes:

1. Executed and imagined opening and closing of both hands
2. Executed and imagined opening and closing of both fists

<div class="alert alert-warning">

We use only one subject (with 90 trials) in this tutorial for demonstration purposes. A more interesting decoding task with many more trials would be to do cross-subject decoding on the same dataset.

</div>

## Load data

You can use any way to load your EEG dataset, here we will use the [python-mne](https://www.martinos.org/mne/stable/index.html) library. For a tutorial from mne using Common Spatial Patterns to decode this data, see [here](http://martinos.org/mne/stable/auto_examples/decoding/plot_decoding_csp_eeg.html).

In [2]:
%%capture
import mne

physionet_paths = mne.datasets.eegbci.load_data(1, [5,6,9,10,13,14])

parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto')
         for path in physionet_paths]

In [3]:
from mne.io import concatenate_raws

raw = concatenate_raws(parts)
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)

Removing orphaned offset at the beginning of the file.
179 events found
Events id: [1 2 3]
90 matching events found
Loading data for 90 events and 497 original time points ...
0 bad epochs dropped


## Convert data to Braindecode Format

Braindecode has a minimalistic ```SignalAndTarget``` class, with attributes `X` for the signal and `y` for the labels. `X` should have these dimensions: trials x channels x timesteps.

In [4]:
import numpy as np
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

We use the first 60 trials for training and the last 30 for testing.

In [5]:
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.torchext.util import set_random_seeds

train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])


## Create the model

Braindecode comes with some predefined convolutional neural network architectures for raw time-domain EEG. here, we use the shallow ConvNet model from [Deep learning with convolutional neural networks for brain mapping and decoding of movement-related information from the human EEG](https://arxiv.org/abs/1703.05051).

In [6]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torchext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=64, n_classes=2, input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

We use [Adam](https://arxiv.org/abs/1412.6980) to optimize the parameters of our network.

In [7]:
from torch import optim

optimizer = optim.Adam(model.parameters())

## Training loop

This is a conventional stochastic gradient descent training loop:
1. Get randomly shuffled batches of trials
2. Compute outputs, loss and gradients on the batches of trials
3. Update your model
4. After iterating through all batches of your dataset, report some statistics like mean accuracy and mean loss.

In [8]:
from braindecode.torchext.util import np_to_var, var_to_np
from braindecode.datautil.iterators import get_balanced_batches
import torch.nn.functional as F
from numpy.random import RandomState
rng = RandomState((2017,6,30))
for i_epoch in range(30):
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=30)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X
        batch_X = train_set.X[i_trials][:,:,:,None]
        batch_y = train_set.y[i_trials]
        net_in = np_to_var(batch_X)
        if cuda:
            net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target.cuda()
        # Remove gradients of last backward pass from all parameters 
        optimizer.zero_grad()
        outputs = model(net_in)
        loss = F.nll_loss(outputs, net_target)
        loss.backward()
        optimizer.step()
    
    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set), ('Test', test_set)):
        net_in = np_to_var(dataset.X[:,:,:,None])
        if cuda:
            net_in.cuda()
        net_target = np_to_var(dataset.y)
        if cuda:
            net_target.cuda()
        outputs = model(net_in)
        loss = F.nll_loss(outputs, net_target)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax(var_to_np(outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))
        

Epoch 0
Train  Loss: 1.36020
Train  Accuracy: 46.7%
Test   Loss: 1.08797
Test   Accuracy: 50.0%
Epoch 1
Train  Loss: 0.70876
Train  Accuracy: 68.3%
Test   Loss: 0.78980
Test   Accuracy: 56.7%
Epoch 2
Train  Loss: 0.48398
Train  Accuracy: 75.0%
Test   Loss: 0.73202
Test   Accuracy: 40.0%
Epoch 3
Train  Loss: 0.51778
Train  Accuracy: 75.0%
Test   Loss: 0.77658
Test   Accuracy: 66.7%
Epoch 4
Train  Loss: 0.51056
Train  Accuracy: 70.0%
Test   Loss: 0.75430
Test   Accuracy: 66.7%
Epoch 5
Train  Loss: 0.42634
Train  Accuracy: 85.0%
Test   Loss: 0.67854
Test   Accuracy: 53.3%
Epoch 6
Train  Loss: 0.61772
Train  Accuracy: 70.0%
Test   Loss: 1.01049
Test   Accuracy: 43.3%
Epoch 7
Train  Loss: 0.51040
Train  Accuracy: 75.0%
Test   Loss: 0.93941
Test   Accuracy: 46.7%
Epoch 8
Train  Loss: 0.25002
Train  Accuracy: 95.0%
Test   Loss: 0.66526
Test   Accuracy: 56.7%
Epoch 9
Train  Loss: 0.31344
Train  Accuracy: 88.3%
Test   Loss: 0.74360
Test   Accuracy: 66.7%
Epoch 10
Train  Loss: 0.26701
Train  Acc

Eventually, we arrive at 63.3% accuracy, so 19 from 30 trials are correctly predicted. See in the next tutorial, how we can achieve better accuracies using cropped training.

## Dataset References


 This dataset was created and contributed to PhysioNet by the developers of the [BCI2000](http://www.schalklab.org/research/bci2000) instrumentation system, which they used in making these recordings. The system is described in:
 
     Schalk, G., McFarland, D.J., Hinterberger, T., Birbaumer, N., Wolpaw, J.R. (2004) BCI2000: A General-Purpose Brain-Computer Interface (BCI) System. IEEE TBME 51(6):1034-1043.

[PhysioBank](https://physionet.org/physiobank/) is a large and growing archive of well-characterized digital recordings of physiologic signals and related data for use by the biomedical research community and further described in:

    Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. (2000) PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220.